# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import re
import pickle
import pandas as pd
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [24]:
# nltk.download(['punkt', 'wordnet'])
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/dungnm31.sc/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/dungnm31.sc/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/dungnm31.sc/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/dungnm31.sc/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/dungnm31.sc/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downl

True

In [18]:
import sqlite3
conn = sqlite3.connect('data/DisasterResponse.db')

df = pd.read_sql(sql="SELECT * FROM EDA", con=conn)
df

,id,message,original,genre,"('related',)","('request',)","('offer',)","('aid_related',)","('medical_help',)","('medical_products',)",...,"('aid_centers',)","('other_infrastructure',)","('weather_related',)","('floods',)","('storm',)","('fire',)","('earthquake',)","('cold',)","('other_weather',)","('direct_report',)"
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('EDA', engine)
X = df.message
y = df[df.columns[4:]]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [20]:
def tokenize(text):
    lemmatizer = WordNetLemmatizer()
    # normalize case and remove punctuation
    text = re.sub(r"[^A-Za-z0-9]",' ',text)

    tokenized_text = word_tokenize(text)

    new_tokens = []
    for token in tokenized_text:
        token = str(token).lower().strip()
        token = lemmatizer.lemmatize(token).strip()
        new_tokens.append(token)

    return new_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [21]:
pipeline = Pipeline([
    ('vt', CountVectorizer(tokenizer=tokenize)),
    ('tf-idf', TfidfTransformer()),
    ('cls', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vt',
                 CountVectorizer(tokenizer=<function tokenize at 0x1280b81f0>)),
                ('tf-idf', TfidfTransformer()),
                ('cls',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [33]:
# from sklearn.base import accuracy_score

def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = (y_pred == y_test).mean()
    print('Accuracy: ', accuracy)
    print('--------------------------------------')
    print('Classification Report:')
    print('--------------------------------------')
    for i in range(10):
        print(y_test.columns[i])
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
        print('---------')

In [34]:
test_model(pipeline, X_test, y_test)

Accuracy:  ('related',)                   0.805615
('request',)                   0.888618
('offer',)                     0.995117
('aid_related',)               0.764266
('medical_help',)              0.917608
('medical_products',)          0.952853
('search_and_rescue',)         0.972994
('security',)                  0.984437
('military',)                  0.969027
('child_alone',)               1.000000
('water',)                     0.946750
('food',)                      0.929204
('shelter',)                   0.930272
('clothing',)                  0.984437
('money',)                     0.976350
('missing_people',)            0.987794
('refugees',)                  0.963381
('death',)                     0.960177
('other_aid',)                 0.865731
('infrastructure_related',)    0.934239
('transport',)                 0.957125
('buildings',)                 0.947513
('electricity',)               0.980012
('tools',)                     0.993439
('hospitals',)               

/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
# List of parameters for this estimator.
pipeline.get_params()

{'memory': None,
 'steps': [('vt',
   CountVectorizer(tokenizer=<function tokenize at 0x1280b81f0>)),
  ('tf-idf', TfidfTransformer()),
  ('cls', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vt': CountVectorizer(tokenizer=<function tokenize at 0x1280b81f0>),
 'tf-idf': TfidfTransformer(),
 'cls': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vt__analyzer': 'word',
 'vt__binary': False,
 'vt__decode_error': 'strict',
 'vt__dtype': numpy.int64,
 'vt__encoding': 'utf-8',
 'vt__input': 'content',
 'vt__lowercase': True,
 'vt__max_df': 1.0,
 'vt__max_features': None,
 'vt__min_df': 1,
 'vt__ngram_range': (1, 1),
 'vt__preprocessor': None,
 'vt__stop_words': None,
 'vt__strip_accents': None,
 'vt__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vt__tokenizer': <function __main__.tokenize(text)>,
 'vt__vocabulary': None,
 'tf-idf__norm': 'l2',
 'tf-idf__smooth_idf': True,
 'tf-idf__sublinear_tf': False,
 'tf-idf__use_idf': True,
 'cls__estimator__boo

In [38]:
parameters = {
    #'tf-idf__use_idf': (True, False),
    'cls__estimator__n_estimators': [50, 100],
    'cls__estimator__min_samples_split': [2, 3, 4],
    #'vt__max_df': (0.5, 0.75, 1.0),
    #'vt__max_features': (None, 5000, 10000),
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2)

In [39]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END cls__estimator__min_samples_split=2, cls__estimator__n_estimators=50; total time= 4.1min
[CV] END cls__estimator__min_samples_split=2, cls__estimator__n_estimators=50; total time= 4.1min


/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END cls__estimator__min_samples_split=2, cls__estimator__n_estimators=50; total time= 4.1min
[CV] END cls__estimator__min_samples_split=2, cls__estimator__n_estimators=50; total time= 4.0min


...

[CV] END cls__estimator__min_samples_split=4, cls__estimator__n_estimators=100; total time= 5.8min


/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END cls__estimator__min_samples_split=4, cls__estimator__n_estimators=100; total time= 5.5min
[CV] END cls__estimator__min_samples_split=4, cls__estimator__n_estimators=100; total time= 5.6min
[CV] END cls__estimator__min_samples_split=4, cls__estimator__n_estimators=100; total time= 5.4min
[CV] END cls__estimator__min_samples_split=4, cls__estimator__n_estimators=100; total time= 4.7min


/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('vt',
                                        CountVectorizer(tokenizer=<function tokenize at 0x1280b81f0>)),
                                       ('tf-idf', TfidfTransformer()),
                                       ('cls',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'cls__estimator__min_samples_split': [2, 3, 4],
                         'cls__estimator__n_estimators': [50, 100]},
             verbose=2)

In [40]:
cv.best_params_

{'cls__estimator__min_samples_split': 3, 'cls__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [41]:
test_model(cv, X_test, y_test)

Accuracy:  ('related',)                   0.803174
('request',)                   0.891974
('offer',)                     0.995117
('aid_related',)               0.772200
('medical_help',)              0.917913
('medical_products',)          0.953158
('search_and_rescue',)         0.973299
('security',)                  0.984437
('military',)                  0.968416
('child_alone',)               1.000000
('water',)                     0.950259
('food',)                      0.929051
('shelter',)                   0.930272
('clothing',)                  0.984437
('money',)                     0.976655
('missing_people',)            0.987946
('refugees',)                  0.963229
('death',)                     0.959414
('other_aid',)                 0.865273
('infrastructure_related',)    0.934391
('transport',)                 0.956210
('buildings',)                 0.948734
('electricity',)               0.979860
('tools',)                     0.993439
('hospitals',)               

/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dungnm31.sc/miniconda3/envs/eda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [42]:
import pickle
pickle.dump(cv, open('cls.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.